<a href="https://colab.research.google.com/github/rpadmanabhan/mdp-rl-project/blob/main/mdp_rl_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stable-baselines3[extra]

     |████████████████████████████████| 177 kB 26.1 MB/s 
     |████████████████████████████████| 1.5 MB 56.5 MB/s 
     |████████████████████████████████| 1.6 MB 52.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616824 sha256=442e5996ad78dd0814fc3b888537b09d7c1b9a804fdf968fbdc3a3a82804cd08
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.4.2-py3-none-any.whl size=441027 sha256=a48eb37abd0ac7247fb0d2b007ace5d6dbf0f43a7d70bebb24c806d64cc5573e
  Stored in directory: /root/.cache/pip/wheels/87/67/2e/6147e7912fe37f5408b80d07527dab807c1d25f5c403a9538a
Successfully built gym AutoROM.accept-rom-license
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.

In [2]:
from dataclasses import dataclass, field
from typing import Dict, List

import statistics

In [3]:
import gym
from gym import spaces

from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.common.cmd_util import make_vec_env


import numpy as np

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/cmd_util.py:6: FutureWarning: Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.
  "Module ``common.cmd_util`` has been renamed to ``common.env_util`` and will be removed in the future.", FutureWarning


In [4]:
# some test data
probe_sequences = [
"..."                   
]
ref_sequences = {
    "AY632535.2 Zika virus strain MR 766, complete genome" : "AGTTGTTGATCTGTGTGAGTCAGACTGCGACAGTTCGAGTCTGAAGCGAGAGCTAACAACAGTATCAACAGGTTTAATTTGGATTTGGAAACGAGAGTTTCTGGTCATGAAAAACCCCAAAGAAGAAATCCGGAGGATCCGGATTGTCAATATGCTAAAACGCGGAGTAGCCCGTGTAAACCCCTTGGGAGGTTTGAAGAGGTTGCCAGCCGGACTTCTGCTGGGTCATGGACCCATCAGAATGGTTTTGGCGATACTAGCCTTTTTGAGATTTACAGCAATCAAGCCATCACTGGGCCTTATCAACAGATGGGGTTCCGTGGGGAAAAAAGAGGCTATGGAAATAATAAAGAAGTTCAAGAAAGATCTTGCTGCCATGTTGAGAATAATCAATGCTAGGAAAGAGAGGAAGAGACGTGGCGCAGACACCAGCATCGGAATCATTGGCCTCCTGCTGACTACAGCCATGGCAGCAGAGATCACTAGACGCGGGAGTGCATACTACATGTACTTGGATAGGAGCGATGCCGGGAAGGCCATTTCGTTTGCTACCACATTGGGAGTGAACAAGTGCCACGTACAGATCATGGACCTCGGGCACATGTGTGACGCCACCATGAGTTATGAGTGCCCTATGCTGGATGAGGGAGTGGAACCAGATGATGTCGATTGCTGGTGCAACACGACATCAACTTGGGTTGTGTACGGAACCTGTCATCACAAAAAAGGTGAGGCACGGCGATCTAGAAGAGCCGTGACGCTCCCTTCTCACTCTACAAGGAAGTTGCAAACGCGGTCGCAGACCTGGTTAGAATCAAGAGAATACACGAAGCACTTGATCAAGGTTGAAAACTGGATATTCAGGAACCCCGGGTTTGCGCTAGTGGCCGTTGCCATTGCCTGGCTTTTGGGAAGCTCGACGAGCCAAAAAGTCATATACTTGGTCATGATACTGCTGATTGCCCCGGCATACAGTATCAGGTGCATTGGAGTCAGCAATAGAGACTTCGTGGAGGGCATGTCAGGTGGGACCTGGGTTGATGTTGTCTTGGAACATGGAGGCTGCGTTACCGTGATGGCACAGGACAAGCCAACAGTCGACATAGAGTTGGTCACGACGACGGTTAGTAACATGGCCGAGGTAAGATCCTATTGCTACGAGGCATCGATATCGGACATGGCTTCGGACAGTCGTTGCCCAACACAAGGTGAAGCCTACCTTGACAAGCAATCAGACACTCAATATGTCTGCAAAAGAACATTAGTGGACAGAGGTTGGGGAAACGGTTGTGGACTTTTTGGCAAAGGGAGCTTGGTGACATGTGCCAAGTTTACGTGTTCTAAGAAGATGACCGGGAAGAGCATTCAACCGGAAAATCTGGAGTATCGGATAATGCTATCAGTGCATGGCTCCCAGCATAGCGGGATGATTGGATATGAAACTGACGAAGATAGAGCGAAAGTCGAGGTTACGCCTAATTCACCAAGAGCGGAAGCAACCTTGGGAGGCTTTGGAAGCTTAGGACTTGACTGTGAACCAAGGACAGGCCTTGACTTTTCAGATCTGTATTACCTGACCATGAACAATAAGCATTGGTTGGTGCACAAAGAGTGGTTTCATGACATCCCATTGCCTTGGCATGCTGGGGCAGACACCGGAACTCCACACTGGAACAACAAAGAGGCATTGGTAGAATTCAAGGATGCCCACGCCAAGAGGCAAACCGTCGTCGTTCTGGGGAGCCAGGAAGGAGCCGTTCACACGGCTCTCGCTGGAGCTCTAGAGGCTGAGATGGATGGTGCAAAGGGAAGGCTGTTCTCTGGCCATTTGAAATGCCGCCTAAAAATGGACAAGCTTAGATTGAAGGGCGTGTCATATTCCTTGTGCACTGCGGCATTCACATTCACCAAGGTCCCAGCTGAAACACTGCATGGAACAGTCACAGTGGAGGTGCAGTATGCAGGGACAGATGGACCCTGCAAGATCCCAGTCCAGATGGCGGTGGACATGCAGACCCTGACCCCAGTTGGAAGGCTGATAACCGCCAACCCCGTGATTACTGAAAGCACTGAGAACTCAAAGATGATGTTGGAGCTTGACCCACCATTTGGGGATTCTTACATTGTCATAGGAGTTGGGGACAAGAAAATCACCCACCACTGGCATAGGAGTGGTAGCACCATCGGAAAGGCATTTGAGGCCACTGTGAGAGGCGCCAAGAGAATGGCAGTCCTGGGGGATACAGCCTGGGACTTCGGATCAGTCGGGGGTGTGTTCAACTCACTGGGTAAGGGCATTCACCAGATTTTTGGAGCAGCCTTCAAATCACTGTTTGGAGGAATGTCCTGGTTCTCACAGATCCTCATAGGCACGCTGCTAGTGTGGTTAGGTTTGAACACAAAGAATGGATCTATCTCCCTCACATGCTTGGCCCTGGGGGGAGTGATGATCTTCCTCTCCACGGCTGTTTCTGCTGACGTGGGGTGCTCAGTGGACTTCTCAAAAAAGGAAACGAGATGTGGCACGGGGGTATTCATCTATAATGATGTTGAAGCCTGGAGGGACCGGTACAAGTACCATCCTGACTCCCCCCGCAGATTGGCAGCAGCAGTCAAGCAGGCCTGGGAAGAGGGGATCTGTGGGATCTCATCCGTTTCAAGAATGGAAAACATCATGTGGAAATCAGTAGAAGGGGAGCTCAATGCTATCCTAGAGGAGAATGGAGTTCAACTGACAGTTGTTGTGGGATCTGTAAAAAACCCCATGTGGAGAGGTCCACAAAGATTGCCAGTGCCTGTGAATGAGCTGCCCCATGGCTGGAAAGCCTGGGGGAAATCGTATTTTGTTAGGGCGGCAAAGACCAACAACAGTTTTGTTGTCGACGGTGACACACTGAAGGAATGTCCGCTTGAGCACAGAGCATGGAATAGTTTTCTTGTGGAGGATCACGGGTTTGGAGTCTTCCACACCAGTGTCTGGCTTAAGGTCAGAGAAGATTACTCATTAGAATGTGACCCAGCCGTCATAGGAACAGCTGTTAAGGGAAGGGAGGCCGCGCACAGTGATCTGGGCTATTGGATTGAAAGTGAAAAGAATGACACATGGAGGCTGAAGAGGGCCCACCTGATTGAGATGAAAACATGTGAATGGCCAAAGTCTCACACATTGTGGACAGATGGAGTAGAAGAAAGTGATCTTATCATACCCAAGTCTTTAGCTGGTCCACTCAGCCACCACAACACCAGAGAGGGTTACAGAACCCAAGTGAAAGGGCCATGGCACAGTGAAGAGCTTGAAATCCGGTTTGAGGAATGTCCAGGCACCAAGGTTTACGTGGAGGAGACATGCGGAACTAGAGGACCATCTCTGAGATCAACTACTGCAAGTGGAAGGGTCATTGAGGAATGGTGCTGTAGGGAATGCACAATGCCCCCACTATCGTTTCGAGCAAAAGACGGCTGCTGGTATGGAATGGAGATAAGGCCCAGGAAAGAACCAGAGAGCAACTTAGTGAGGTCAATGGTGACAGCGGGGTCAACCGATCATATGGACCACTTCTCTCTTGGAGTGCTTGTGATTCTACTCATGGTGCAGGAGGGGTTGAAGAAGAGAATGACCACAAAGATCATCATGAGCACATCAATGGCAGTGCTGGTAGTCATGATCTTGGGAGGATTTTCAATGAGTGACCTGGCCAAGCTTGTGATCCTGATGGGTGCTACTTTCGCAGAAATGAACACTGGAGGAGATGTAGCTCACTTGGCATTGGTAGCGGCATTTAAAGTCAGACCAGCCTTGCTGGTCTCCTTCATTTTCAGAGCCAATTGGACACCCCGTGAGAGCATGCTGCTAGCCCTGGCTTCGTGTCTTCTGCAAACTGCGATCTCTGCTCTTGAAGGTGACTTGATGGTCCTCATTAATGGATTTGCTTTGGCCTGGTTGGCAATTCGAGCAATGGCCGTGCCACGCACTGACAACATCGCTCTACCAATCTTGGCTGCTCTAACACCACTAGCTCGAGGCACACTGCTCGTGGCATGGAGAGCGGGCCTGGCTACTTGTGGAGGGATCATGCTCCTCTCCCTGAAAGGGAAAGGTAGTGTGAAGAAGAACCTGCCATTTGTCATGGCCCTGGGATTGACAGCTGTGAGGGTAGTAGACCCTATTAATGTGGTAGGACTACTGTTACTCACAAGGAGTGGGAAGCGGAGCTGGCCCCCTAGTGAAGTTCTCACAGCCGTTGGCCTGATATGTGCACTGGCCGGAGGGTTTGCCAAGGCAGACATTGAGATGGCTGGACCCATGGCTGCAGTAGGCTTGCTAATTGTCAGCTATGTGGTCTCGGGAAAGAGTGTGGACATGTACATTGAAAGAGCAGGTGACATCACATGGGAAAAGGACGCGGAAGTCACTGGAAACAGTCCTCGGCTTGACGTGGCACTGGATGAGAGTGGTGACTTCTCCTTGGTAGAGGAAGATGGTCCACCCATGAGAGAGATCATACTCAAGGTGGTCCTGATGGCCATCTGTGGCATGAACCCAATAGCTATACCTTTTGCTGCAGGAGCGTGGTATGTGTATGTGAAGACTGGGAAAAGGAGTGGCGCCCTCTGGGACGTGCCTGCTCCCAAAGAAGTGAAGAAAGGAGAGACCACAGATGGAGTGTACAGAGTGATGACTCGCAGACTGCTAGGTTCAACACAGGTTGGAGTGGGAGTCATGCAAGAGGGAGTCTTCCACACCATGTGGCACGTTACAAAAGGAGCCGCACTGAGGAGCGGTGAGGGAAGACTTGATCCATACTGGGGGGATGTCAAGCAGGACTTGGTGTCATACTGTGGGCCTTGGAAGTTGGATGCAGCTTGGGATGGACTCAGCGAGGTACAGCTTTTGGCCGTACCTCCCGGAGAGAGGGCCAGAAACATTCAGACCCTGCCTGGAATATTCAAGACAAAGGACGGGGACATCGGAGCAGTTGCTCTGGACTACCCTGCAGGGACCTCAGGATCTCCGATCCTAGACAAATGTGGAAGAGTGATAGGACTCTATGGCAATGGGGTTGTGATCAAGAATGGAAGCTATGTTAGTGCTATAACCCAGGGAAAGAGGGAGGAGGAGACTCCGGTTGAATGTTTCGAACCCTCGATGCTGAAGAAGAAGCAGCTAACTGTCTTGGATCTGCATCCAGGAGCCGGAAAAACCAGGAGAGTTCTTCCTGAAATAGTCCGTGAAGCCATAAAAAAGAGACTCCGGACAGTGATCTTGGCACCAACTAGGGTTGTCGCTGCTGAGATGGAGGAGGCCTTGAGAGGACTTCCGGTGCGTTACATGACAACAGCAGTCAACGTCACCCATTCTGGGACAGAAATCGTTGATTTGATGTGCCATGCCACTTTCACTTCACGCTTACTACAACCCATCAGAGTCCCTAATTACAATCTCAACATCATGGATGAAGCCCACTTCACAGACCCCTCAAGTATAGCTGCAAGAGGATACATATCAACAAGGGTTGAAATGGGCGAGGCGGCTGCCATTTTTATGACTGCCACACCACCAGGAACCCGTGATGCGTTTCCTGACTCTAACTCACCAATCATGGACACAGAAGTGGAAGTCCCAGAGAGAGCCTGGAGCTCAGGCTTTGATTGGGTGACAGACCATTCTGGGAAAACAGTTTGGTTCGTTCCAAGCGTGAGAAACGGAAATGAAATCGCAGCCTGTCTGACAAAGGCTGGAAAGCGGGTCATACAGCTCAGCAGGAAGACTTTTGAGACAGAATTTCAGAAAACAAAAAATCAAGAGTGGGACTTTGTCATAACAACTGACATCTCAGAGATGGGCGCCAACTTCAAGGCTGACCGGGTCATAGACTCTAGGAGATGCCTAAAACCAGTCATACTTGATGGTGAGAGAGTCATCTTGGCTGGGCCCATGCCTGTCACGCATGCTAGTGCTGCTCAGAGGAGAGGACGTATAGGCAGGAACCCTAACAAACCTGGAGATGAGTACATGTATGGAGGTGGGTGTGCAGAGACTGATGAAGGCCATGCACACTGGCTTGAAGCAAGAATGCTTCTTGACAACATCTACCTCCAGGATGGCCTCATAGCCTCGCTCTATCGGCCTGAGGCCGATAAGGTAGCCGCCATTGAGGGAGAGTTTAAGCTGAGGACAGAGCAAAGGAAGACCTTCGTGGAACTCATGAAGAGAGGAGACCTTCCCGTCTGGCTAGCCTATCAGGTTGCATCTGCCGGAATAACTTACACAGACAGAAGATGGTGCTTTGATGGCACAACCAACAACACCATAATGGAAGACAGTGTACCAGCAGAGGTTTGGACAAAGTATGGAGAGAAGAGAGTGCTCAAACCGAGATGGATGGATGCTAGGGTCTGTTCAGACCATGCGGCCCTGAAGTCGTTCAAAGAATTCGCCGCTGGAAAAAGAGGAGCGGCTTTGGGAGTAATGGAGGCCCTGGGAACACTGCCAGGACACATGACAGAGAGGTTTCAGGAAGCCATTGACAACCTCGCCGTGCTCATGCGAGCAGAGACTGGAAGCAGGCCTTATAAGGCAGCGGCAGCCCAACTGCCGGAGACCCTAGAGACCATTATGCTCTTAGGTTTGCTGGGAACAGTTTCACTGGGGATCTTCTTCGTCTTGATGCGGAATAAGGGCATCGGGAAGATGGGCTTTGGAATGGTAACCCTTGGGGCCAGTGCATGGCTCATGTGGCTTTCGGAAATTGAACCAGCCAGAATTGCATGTGTCCTCATTGTTGTGTTTTTATTACTGGTGGTGCTCATACCCGAGCCAGAGAAGCAAAGATCTCCCCAAGATAACCAGATGGCAATTATCATCATGGTGGCAGTGGGCCTTCTAGGTTTGATAACTGCAAACGAACTTGGATGGCTGGAAAGAACAAAAAATGACATAGCTCATCTAATGGGAAGGAGAGAAGAAGGAGCAACCATGGGATTCTCAATGGACATTGATCTGCGGCCAGCCTCCGCCTGGGCTATCTATGCCGCATTGACAACTCTCATCACCCCAGCTGTCCAACATGCGGTAACCACTTCATACAACAACTACTCCTTAATGGCGATGGCCACACAAGCTGGAGTGCTGTTTGGCATGGGCAAAGGGATGCCATTTATGCATGGGGACCTTGGAGTCCCGCTGCTAATGATGGGTTGCTATTCACAATTAACACCCCTGACTCTGATAGTAGCTATCATTCTGCTTGTGGCGCACTACATGTACTTGATCCCAGGCCTACAAGCGGCAGCAGCGCGTGCTGCCCAGAAAAGGACAGCAGCTGGCATCATGAAGAATCCCGTTGTGGATGGAATAGTGGTAACTGACATTGACACAATGACAATAGACCCCCAGGTGGAGAAGAAGATGGGACAAGTGTTACTCATAGCAGTAGCCATCTCCAGTGCTGTGCTGCTGCGGACCGCCTGGGGATGGGGGGAGGCTGGAGCTCTGATCACAGCAGCGACCTCCACCTTGTGGGAAGGCTCTCCAAACAAATACTGGAACTCCTCTACAGCCACCTCACTGTGCAACATCTTCAGAGGAAGCTATCTGGCAGGAGCTTCCCTTATCTATACAGTGACGAGAAACGCTGGCCTGGTTAAGAGACGTGGAGGTGGGACGGGAGAGACTCTGGGAGAGAAGTGGAAAGCTCGTCTGAATCAGATGTCGGCCCTGGAGTTCTACTCTTATAAAAAGTCAGGTATCACTGAAGTGTGTAGAGAGGAGGCTCGCCGTGCCCTCAAGGATGGAGTGGCCACAGGAGGACATGCCGTATCCCGGGGAAGTGCAAAGATCAGATGGTTGGAGGAGAGAGGATATCTGCAGCCCTATGGGAAGGTTGTTGACCTCGGATGTGGCAGAGGGGGCTGGAGCTATTATGCCGCCACCATCCGCAAAGTGCAGGAGGTGAGAGGATACACAAAGGGAGGTCCCGGTCATGAAGAACCCATGCTGGTGCAAAGCTATGGGTGGAACATAGTTCGTCTCAAGAGTGGAGTGGACGTCTTCCACATGGCGGCTGAGCCGTGTGACACTCTGCTGTGTGACATAGGTGAGTCATCATCTAGTCCTGAAGTGGAAGAGACACGAACACTCAGAGTGCTCTCTATGGTGGGGGACTGGCTTGAAAAAAGACCAGGGGCCTTCTGTATAAAGGTGCTGTGCCCATACACCAGCACTATGATGGAAACCATGGAGCGACTGCAACGTAGGCATGGGGGAGGATTAGTCAGAGTGCCATTGTGTCGCAACTCCACACATGAGATGTACTGGGTCTCTGGGGCAAAGAGCAACATCATAAAAAGTGTGTCCACCACAAGTCAGCTCCTCCTGGGACGCATGGATGGCCCCAGGAGGCCAGTGAAATATGAGGAGGATGTGAACCTCGGCTCGGGTACACGAGCTGTGGCAAGCTGTGCTGAGGCTCCTAACATGAAAATCATCGGCAGGCGCATTGAGAGAATCCGCAATGAACATGCAGAAACATGGTTTCTTGATGAAAACCACCCATACAGGACATGGGCCTACCATGGGAGCTACGAAGCCCCCACGCAAGGATCAGCGTCTTCCCTCGTGAACGGGGTTGTTAGACTCCTGTCAAAGCCTTGGGACGTGGTGACTGGAGTTACAGGAATAGCCATGACTGACACCACACCATACGGCCAACAAAGAGTCTTCAAAGAAAAAGTGGACACCAGGGTGCCAGATCCCCAAGAAGGCACTCGCCAGGTAATGAACATAGTCTCTTCCTGGCTGTGGAAGGAGCTGGGGAAACGCAAGCGGCCACGCGTCTGCACCAAAGAAGAGTTTATCAACAAGGTGCGCAGCAATGCAGCACTGGGAGCAATATTTGAAGAGGAAAAAGAATGGAAGACGGCTGTGGAAGCTGTGAATGATCCAAGGTTTTGGGCCCTAGTGGATAGGGAGAGAGAACACCACCTGAGAGGAGAGTGTCACAGCTGTGTGTACAACATGATGGGAAAAAGAGAAAAGAAGCAAGGAGAGTTCGGGAAAGCAAAAGGTAGCCGCGCCATCTGGTACATGTGGTTGGGAGCCAGATTCTTGGAGTTTGAAGCCCTTGGATTCTTGAACGAGGACCATTGGATGGGAAGAGAAAACTCAGGAGGTGGAGTCGAAGGGTTAGGATTGCAAAGACTTGGATACATTCTAGAAGAAATGAATCGGGCACCAGGAGGAAAGATGTACGCAGATGACACTGCTGGCTGGGACACCCGCATTAGTAAGTTTGATCTGGAGAATGAAGCTCTGATTACCAACCAAATGGAGGAAGGGCACAGAACTCTGGCGTTGGCCGTGATTAAATACACATACCAAAACAAAGTGGTGAAGGTTCTCAGACCAGCTGAAGGAGGAAAAACAGTTATGGACATCATTTCAAGACAAGACCAGAGAGGGAGTGGACAAGTTGTCACTTATGCTCTCAACACATTCACCAACTTGGTGGTGCAGCTTATCCGGAACATGGAAGCTGAGGAAGTGTTAGAGATGCAAGACTTATGGTTGTTGAGGAAGCCAGAGAAAGTGACCAGATGGTTGCAGAGCAATGGATGGGATAGACTCAAACGAATGGCGGTCAGTGGAGATGACTGCGTTGTGAAGCCAATCGATGATAGGTTTGCACATGCCCTCAGGTTCTTGAATGACATGGGAAAAGTTAGGAAAGACACACAGGAGTGGAAACCCTCGACTGGATGGAGCAATTGGGAAGAAGTCCCGTTCTGCTCCCACCACTTCAACAAGCTGTACCTCAAGGATGGGAGATCCATTGTGGTCCCTTGCCGCCACCAAGATGAACTGATTGGCCGAGCTCGCGTCTCACCAGGGGCAGGATGGAGCATCCGGGAGACTGCCTGTCTTGCAAAATCATATGCGCAGATGTGGCAGCTCCTTTATTTCCACAGAAGAGACCTTCGACTGATGGCTAATGCCATTTGCTCGGCTGTGCCAGTTGACTGGGTACCAACTGGGAGAACCACCTGGTCAATCCATGGAAAGGGAGAATGGATGACCACTGAGGACATGCTCATGGTGTGGAATAGAGTGTGGATTGAGGAGAACGACCATATGGAGGACAAGACTCCTGTAACAAAATGGACAGACATTCCCTATCTAGGAAAAAGGGAGGACTTATGGTGTGGATCCCTTATAGGGCACAGACCCCGCACCACTTGGGCTGAAAACATCAAAGACACAGTCAACATGGTGCGCAGGATCATAGGTGATGAAGAAAAGTACATGGACTATCTATCCACCCAAGTCCGCTACTTGGGTGAGGAAGGGTCCACACCCGGAGTGTTGTAAGCACCAATTTTAGTGTTGTCAGGCCTGCTAGTCAGCCACAGTTTGGGGAAAGCTGTGCAGCCTGTAACCCCCCCAGGAGAAGCTGGGAAACCAAGCTCATAGTCAGGCCGAGAACGCCATGGCACGGAAGAAGCCATGCTGCCTGTGAGCCCCTCAGAGGACACTGAGTCAAAAAACCCCACGCGCTTGGAAGCGCAGGATGGGAAAAGAAGGTGGCGACCTTCCCCACCCTTCAATCTGGGGCCTGAACTGGAGACTAGCTGTGAATCTCCAGCAGAGGGACTAGTGGTTAGAGGAGACCCCCCGGAAAACGCAAAACAGCATATTGACGTGGGAAAGACCAGAGACTCCATGAGTTTCCACCACGCTGGCCGCCAGGCACAGATCGCCGAACTTCGGCGGCCGGTGTGGGGAAATCCATGGTTTCT"
}
print("Number of initial Probe candidates: ", len(probe_sequences))
print("Reference Sequence Length(s): ", [(seq_name, len(seq)) for seq_name, seq in ref_sequences.items()])

Number of initial Probe candidates:  215
Reference Sequence Length(s):  [('AY632535.2 Zika virus strain MR 766, complete genome', 10794)]


In [5]:
@dataclass(frozen = True)
class ProbeOligo:
    seq: str
    hyb_extension: int = 50

    def ret_coverage_range(self, ref_seq):
        start = ref_seq.find(self.seq)
        assert start != -1
        # return coverage [start, end) range in reference sequence
        return start - self.hyb_extension, start + len(self.seq) + self.hyb_extension

@dataclass()
class RefSeqGroup:
    ref_sequences: Dict[str, str]

    def __post_init__(self):
        self.coverage_profile = {seq_name: np.zeros(len(seq), dtype = np.int16) for seq_name, seq in self.ref_sequences.items()}

    def _reset_coverage_profile(self):
        """
        """
        self.coverage_profile = {seq_name: np.zeros(len(seq), dtype = np.int16) for seq_name, seq in self.ref_sequences.items()}

    def update_coverage_profile(self, probe: ProbeOligo):
        """ Update base level coverage for all reference sequences
        """
        for seq_name,seq in self.ref_sequences.items():
            start, end = probe.ret_coverage_range(seq)
            # need some bounds because of hyb_extension param for a ProbeOligo
            start = max(0, start)
            end = min(len(seq), end)
            for idx in range(start, end):
                self.coverage_profile[seq_name][idx] += 1

    def ret_num_bases_covered(self):
        """ Provide a count for the number of bases covered for each reference sequence
        """
        num_bases_covered = []
        for seq_name, seq in self.ref_sequences.items():
            num_bases_covered.append(np.count_nonzero(self.coverage_profile[seq_name]))
        return num_bases_covered

    def ret_mean_bases_covered(self):
        """Return mean bases covered across all reference sequences
        """
        return statistics.mean(self.ret_num_bases_covered())


class OligoEnv(gym.Env):
    def __init__(self, *args, ref_seq_group: RefSeqGroup, oligos: List[ProbeOligo], **kwargs):
        super(OligoEnv, self).__init__()

        # initialize group of ref sequences
        self.ref_seq_group = ref_seq_group
        self.total_ref_seq_len = sum([len(seq) for seq_name, seq in self.ref_seq_group.ref_sequences.items()])

        # threshold for coverage across reference sequence group
        self.coverage_threshold = kwargs.get("coverage_threshold", 1.0)

        # list of candidate oligos we have
        self.oligos = oligos

        # where is the agent now ?
        self.chosen_oligos = []

        # action space
        self.action_space = spaces.Discrete(len(self.oligos))

        # observation space - state of the agent
        self.observation_space = spaces.MultiBinary(len(self.oligos))

        # some reward related constants
        self._reward_oligo_penalty = -1
        self._reward_action_masking = -50

        # variables to reset for state space
        self._actions_taken = None

    def _check_termination(self, observation):
        # terminate if we hit the desired coverage threshold
        return self.ref_seq_group.ret_mean_bases_covered() / self.total_ref_seq_len >= self.coverage_threshold

    def _compute_reward(self, action):
        oligo = self.oligos[action]
        cur_bases_covered = self.ref_seq_group.ret_mean_bases_covered()
        # update coverage profile
        self.ref_seq_group.update_coverage_profile(self.oligos[action])
        now_bases_covered = self.ref_seq_group.ret_mean_bases_covered()
        # delayed reward when we end, for now just penalize this oligo
        return self._reward_oligo_penalty + now_bases_covered - cur_bases_covered

    def step(self, action):
        """
        """
        # we do not want to repeat an oligo so we want to have some mechanism for penalizing this
        reward = self._reward_action_masking
        if action not in self._actions_taken:
            # keep track of oligos added
            self._actions_taken.add(action)
            # compute reward
            reward = self._compute_reward(action)

        # which oligos have been picked
        observation = np.array([1 if idx in self._actions_taken else 0 for idx in range(0, len(self.oligos))], dtype = np.int8)

        # have we met stopping criteria
        done = bool(self._check_termination(observation))

        # we accumulate delayed reward at the end
        #if done:
        #    reward = self.ref_seq_group.ret_mean_bases_covered()

        # not much info to add currently
        info = {}

        return observation, reward, done, info

    def reset(self):

        # no oligos picked
        self._actions_taken = set({})
        init_observation = np.zeros(len(self.oligos), dtype = np.int8)
        self.ref_seq_group._reset_coverage_profile()

        return init_observation

    def render(self, mode = "console"):
        # To Do: Add some neat visualization of the oligo bindings to reference
        pass

    def close (self):
        pass

In [6]:
# init Probe and Reference datastructures
oligos = [ProbeOligo(probe_seq) for probe_seq in probe_sequences]
ref_seq_group = RefSeqGroup(ref_sequences)

In [7]:
# init custom environment
env = OligoEnv(ref_seq_group=ref_seq_group, oligos = oligos)

In [8]:
# check that the environment works
check_env(env)


In [9]:
# learn an optimal policy
env_wrap = make_vec_env(lambda: env, n_envs = 1)


In [10]:
model = A2C("MlpPolicy", env, verbose = 1).learn(10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 243      |
|    ep_rew_mean        | 6.29e+03 |
| time/                 |          |
|    fps                | 549      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -5.32    |
|    explained_variance | 0        |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -833     |
|    value_loss         | 2.66e+04 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 288      |
|    ep_rew_mean        | 4.56e+03 |
| time/                 |          |
|    fps                | 566      |
|    iterations         | 200      |
|    time_elapsed 

In [11]:
# see how it performs
obs = env.reset()
n_steps = 1000
actions = set({})
for step in range(n_steps):
    action, _ = model.predict(obs, deterministic=False)
    print(step, action)
    observation, reward, done, info = env.step(action)
    actions.add(action)
    if done:
      break


0 156
1 49
2 104
3 127
4 3
5 57
6 74
7 179
8 94
9 59
10 141
11 165
12 108
13 97
14 3
15 171
16 160
17 27
18 107
19 162
20 67
21 33
22 152
23 103
24 8
25 176
26 187
27 194
28 144
29 181
30 16
31 88
32 175
33 138
34 181
35 0
36 168
37 202
38 14
39 142
40 80
41 128
42 75
43 138
44 16
45 23
46 184
47 40
48 158
49 8
50 96
51 154
52 204
53 178
54 209
55 10
56 90
57 5
58 171
59 2
60 49
61 203
62 188
63 104
64 55
65 200
66 200
67 24
68 91
69 197
70 80
71 129
72 13
73 198
74 41
75 83
76 88
77 79
78 100
79 184
80 126
81 56
82 36
83 128
84 148
85 210
86 11
87 169
88 37
89 204
90 133
91 101
92 151
93 113
94 191
95 186
96 96
97 26
98 179
99 23
100 5
101 111
102 207
103 42
104 47
105 127
106 201
107 77
108 163
109 17
110 21
111 81
112 205
113 97
114 70
115 167
116 81
117 164
118 95
119 146
120 19
121 125
122 208
123 33
124 199
125 128
126 177
127 139
128 99
129 71
130 28
131 57
132 37
133 115
134 151
135 107
136 108
137 118
138 1
139 15
140 157
141 192
142 102
143 11
144 18
145 50
146 66
147 124
148

In [12]:
print(actions)
print(len(actions))

{0, 1, 2, 3, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 26, 27, 28, 29, 31, 33, 36, 37, 38, 40, 41, 42, 44, 47, 49, 50, 52, 53, 54, 55, 56, 57, 59, 60, 62, 66, 67, 69, 70, 71, 72, 73, 74, 75, 77, 79, 80, 81, 82, 83, 84, 87, 88, 89, 90, 91, 92, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 110, 111, 112, 113, 115, 116, 118, 119, 123, 124, 125, 126, 127, 128, 129, 130, 133, 134, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 151, 152, 154, 156, 157, 158, 159, 160, 162, 163, 164, 165, 167, 168, 169, 170, 171, 174, 175, 176, 177, 178, 179, 180, 181, 182, 184, 186, 187, 188, 190, 191, 192, 194, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 214}
161


In [16]:
with open("picked_probes_rl.fasta", "w") as out_handle:
    for idx in actions:
        out_handle.write(f">seq{idx}\n")
        out_handle.write(probe_sequences[idx])
        out_handle.write("\n")

In [22]:
env.ref_seq_group.coverage_profile

{'AY632535.2 Zika virus strain MR 766, complete genome': array([1, 1, 1, ..., 0, 0, 0], dtype=int16)}

In [250]:
test_oligos = ["AGTTGTTGATCTGTGTGAGTCAG", "TGTGAGTCAGACTG", "CTGCGACAGTTCGAGTCTGAAGCGAGAGCTAACAAC", "AGTTGTTGATCTGTGTGAGTCAGACTGC", "GACAGTTCGAGTCTGAAGCGAGAGCTAACAA", "CAGTATCAACAGGTTTAATTTGGATTTGGAAACGAGAGTTTCTGGTCATGAAAAA"]
test_seqs = {"test_seq": "AGTTGTTGATCTGTGTGAGTCAGACTGCGACAGTTCGAGTCTGAAGCGAGAGCTAACAACAGTATCAACAGGTTTAATTTGGATTTGGAAACGAGAGTTTCTGGTCATGAAAAA"}

In [253]:
# init Probe and Reference datastructures
oligos = [ProbeOligo(probe_seq, hyb_extension=0) for probe_seq in test_oligos]
ref_seq_group = RefSeqGroup(test_seqs)

In [255]:
# init custom environment
env = OligoEnv(ref_seq_group=ref_seq_group, oligos = oligos)
check_env(env)
model = A2C("MlpPolicy", env, verbose = 1).learn(10000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 6.37     |
|    ep_rew_mean        | 4.6      |
| time/                 |          |
|    fps                | 569      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.71    |
|    explained_variance | 0.0635   |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -137     |
|    value_loss         | 6.44e+03 |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 4.7      |
|    ep_rew_mean        | 75       |
| time/                 |          |
|    fps                | 588      |
|    iterations         | 200      |
|    time_elapsed 

In [256]:
# see how it performs
obs = env.reset()
n_steps = 100
actions = set({})
for step in range(n_steps):
    action, other_ = model.predict(obs, deterministic=True)
    print(step, action, other_)
    out = env.step(action)
    print(out)
    actions.add(action)

0 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), 54, False, {})
1 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
2 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
3 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
4 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
5 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
6 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
7 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
8 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
9 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
10 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
11 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
12 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
13 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
14 5 None
(array([0, 0, 0, 0, 0, 1], dtype=int8), -50, False, {})
15 5 None
(array([0, 